In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

from time import time
import pandas as pd
import time as t_
import re
import requests
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
import xlrd


from google.colab import drive
drive.mount('/content/drive')


class Main():
    def __init__(self):
        TOI_ARCH_URL = "https://timesofindia.indiatimes.com/archive.cms"
        response = requests.get(TOI_ARCH_URL)
        if response.status_code == 200:
            self.SOUP = BeautifulSoup(response.content, 'lxml')
        else:
            raise Exception('Cannot reach the website.')

    def fetching_month_urls(self):
        start1 = time()
        workbook = Workbook()
        sheet = workbook.active
        sheet['A1'] = 'ORGINAL URL'
        sheet['B1'] = 'MONTH URL'
        sheet['C1'] = 'TIME'
        count = 2

        month_urls = []
        for tag in self.SOUP.find_all('a', class_='normtxt'):
            month_urls.append("https://timesofindia.indiatimes.com" + tag.attrs['href'])
            sheet['A' + str(count)] = "https://timesofindia.indiatimes.com/archive.cms"
            sheet['B' + str(count)] = "https://timesofindia.indiatimes.com" + tag.attrs['href']
            sheet['C' + str(count)] = str( time()-start1 )
            count += 1
            
        workbook.save('/content/drive/My Drive/'+ 'fetching_month_urls' + '.xlsx')
        print("fetching month urls done")
        return month_urls

    def fetching_days_in_month_urls(self):
        start1 = time()
        workbook = Workbook()
        sheet = workbook.active
        sheet['A1'] = 'ORIGINAL URL'
        sheet['B1'] = 'DAY URL'
        sheet['C1'] = 'TIME'
        count = 2
        month_urls = self.reading_excel("fetching_month_urls.xlsx")
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--log-level=3')
        #driver = webdriver.Chrome("C:\Program Files (x86)\chromedriver.exe", chrome_options=options)
        driver = webdriver.Chrome('chromedriver',options=options)


        for url in month_urls:
            date_link_dict = {}
            #print(url)
            driver.get(url)
            temp_year = driver.find_element_by_xpath("//tbody/tr[2]/td[1]/div[2]/a[2]")
            temp_month = driver.find_element_by_xpath("//tbody/tr[2]/td[1]/div[2]/b[1]")
            table = driver.find_element_by_id("calender")
            temp1 = table.find_element_by_tag_name("tbody")
            temp2 = temp1.find_elements_by_tag_name("tr")
            for temp in temp2:
                temp3 = temp.find_elements_by_tag_name("td")
                for in_cal in temp3:
                    try:
                        if int(in_cal.text):
                            links = in_cal.find_elements_by_tag_name("a")
                            for link in links:
                                if link.get_attribute("href") != "":
                                    sheet['A' + str(count)] = url
                                    sheet['B' + str(count)] = link.get_attribute("href")
                                    sheet['C' + str(count)] = str( time()-start1 )
                                    count += 1
                    except Exception as err:
                        pass
                        
        driver.quit()
        print("fetching_days_in_month_urls completed.")
        workbook.save('/content/drive/My Drive/'+ 'fetching_days_in_month_urls2' + '.xlsx')


    def reading_excel(self, filename):
        all_urls = []
        wb = xlrd.open_workbook('/content/drive/My Drive/'+ filename)
        sheet = wb.sheet_by_index(0)
        no_rows = sheet.nrows
        print(no_rows)
        #print(sheet.cell_value(0, 0))
        for i in range(no_rows-1):
            #print("_"+sheet.cell_value(i, 0))
            url = sheet.cell_value(i+1, 1)
            all_urls.append(url)
        return all_urls



    def fetch_para_page_link(self):
        file_status = open('/content/drive/My Drive/'+'Status.txt', 'a')
        start1 = time()
        month_urls = self.reading_excel('fetching_days_in_month_urls2.xlsx')
        error_occurred = False

        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--log-level=3')
        driver = webdriver.Chrome('chromedriver',options=options)
        len_count = 0
        month_urls_len = len(month_urls)


        #for url in month_urls:
        for i in range(month_urls_len):
            if i < 3600:
                url = month_urls[i]
                error_occurred = False
                if len_count%100 == 0:
                  print(len_count, time()-start1)
                while error_occurred == False:
                    try:
                        driver.get(url)

                        tbody = driver.find_elements_by_xpath("/html[1]/body[1]/div[1]/table[2]/tbody[1]/tr[2]/td[1]/div[3]/table[1]/tbody[1]")
                        tr = tbody[0].find_elements_by_tag_name("tr")
                        for tr_ in tr:
                            span = tr_.find_elements_by_tag_name("span")
                            for span_ in span:
                                links_list = span_.find_elements_by_tag_name("a")
                                for link in links_list:
                                    #print("_" + link.get_attribute("href"))
                                    '''sheet['A' + str(count)] = url
                                    sheet['B' + str(count)] = link.get_attribute("href")
                                    sheet['C' + str(count)] = str( time()-start1 )
                                    count += 1'''
                                    file_status.write(url + "\t" + link.get_attribute("href") + "\t" + "\n")
                        error_occurred = True

                    except Exception as err:
                        error_occurred = False
                        driver.quit()
                        options = webdriver.ChromeOptions()
                        options.add_argument('--headless')
                        options.add_argument('--no-sandbox')
                        options.add_argument('--disable-dev-shm-usage')
                        options.add_argument('--ignore-certificate-errors')
                        options.add_argument('--incognito')
                        options.add_argument('--log-level=3')
                        driver = webdriver.Chrome('chromedriver',options=options)
                        print("error occured.", err)

                len_count += 1
            else:
              break          

        #workbook.save('/content/drive/My Drive/'+ 'fetching_days_in_month_urls3' + '.xlsx')
        file_status.close()


    def fetch_para(self):
        df1 = pd.read_csv('/content/drive/My Drive/Link_full.txt', sep = '\t')
        df1.columns = ['ALL_LINK_PAGE', 'PARA_PAGE_LINK']

        print(df1.shape)
        len_link1 = df1.shape[0]

        file_status = open('/content/drive/My Drive/'+'Business_paragraph.txt', 'a')
        file_new_link = open('/content/drive/My Drive/'+'no_business_structure.txt', 'a')
        start1 = time()

        for i in range(len_link1):
            if i%1000 == 0:
                print( (len_link1-i), (time()-start1))
            url = df1["PARA_PAGE_LINK"][i]
            #print(i, "_"+url)
            response = requests.get(url)
            if response.status_code == 200:
                SOUP = BeautifulSoup(response.content, 'lxml')
                para_list = SOUP.find_all("div", class_= "ga-headlines")
                if len(para_list)>0:
                  heading = SOUP.find_all("h1", class_ = "_23498")
                  date = SOUP.find_all("div", class_ = "_3Mkg- byline")
                  file_status.write(url + "\t" + date[0].text + "\t" + heading[0].text + "\t" + para_list[0].text + "\n")
                else:
                  file_new_link.write(df1['ALL_LINK_PAGE'][i] + url + "\n")
            else:
              pass
        file_status.close()
        file_new_link.close()


main = Main()
#main.fetching_month_urls()
#main.fetching_days_in_month_urls()
#main.fetch_para_page_link()
main.fetch_para()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chro